In [206]:
import os
import csv
import pandas as pd

# Import SQL Alchemy
from sqlalchemy import create_engine, inspect

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

In [207]:
# Create a reference for clean stations file
csv_path = "CSV_cleaned/stations_clean.csv"

# Read the CSV into a Pandas DataFrame
stations_df = pd.read_csv(csv_path)

stations_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [208]:
# Create a reference for clean stations file
csv_path = "CSV_cleaned/measure_clean.csv"

# Read the CSV into a Pandas DataFrame
measure_df = pd.read_csv(csv_path)

measure_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [209]:
# Create the measurement class

class Measurement(Base):
    __tablename__ = 'measurement'
    __table_args__ = {'sqlite_autoincrement': True}
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs=Column(Integer)


In [210]:
# Create the stations class
class Stations(Base):
    __tablename__ = 'stations'
    __table_args__ = {'sqlite_autoincrement': True}
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude=Column(Float)
    elevation=Column(Float)


In [215]:
# Create a connection to a SQLite database
### BEGIN SOLUTION
engine = create_engine('sqlite:///surfsup.sqlite')
engine1 = create_engine('sqlite:///surfsup.sqlite')
### END SOLUTION

In [216]:
# Create the garbage_collection table within the database
Base.metadata.create_all(engine)

In [217]:
# To push the objects made and query the server we use a Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)


In [218]:
# Insert pandas dataframe to database tables
# measure_df.to_sql(name='measurement', con=engine, index=False,if_exists='replace')
measure_df.to_sql(con=engine, index_label='id', name=Measurement.__tablename__, if_exists='replace')

In [219]:
# Use `engine.execute` to select and display the first 10 rows from the emoji table
engine.execute('SELECT * FROM measurement LIMIT 10').fetchall()

[(0, 'USC00519397', '2010-01-01', 0.08, 65),
 (1, 'USC00519397', '2010-01-02', 0.0, 63),
 (2, 'USC00519397', '2010-01-03', 0.0, 74),
 (3, 'USC00519397', '2010-01-04', 0.0, 76),
 (4, 'USC00519397', '2010-01-07', 0.06, 70),
 (5, 'USC00519397', '2010-01-08', 0.0, 64),
 (6, 'USC00519397', '2010-01-09', 0.0, 68),
 (7, 'USC00519397', '2010-01-10', 0.0, 73),
 (8, 'USC00519397', '2010-01-11', 0.01, 64),
 (9, 'USC00519397', '2010-01-12', 0.0, 61)]

In [220]:
stations_df.to_sql(con=engine, index_label='id', name='stations', if_exists='replace')

In [221]:
engine.execute('SELECT * FROM stations LIMIT 10').fetchall()

[(0, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (1, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (2, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (3, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (4, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 (5, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 (6, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 (7, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 (8, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [222]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [223]:
# Collect the names of tables within the database
inspector.get_table_names()

['measurement', 'stations']

In [224]:
columns = inspector.get_columns('stations')
for column in columns:
    print(column["name"])

id
station
name
latitude
longitude
elevation


In [225]:
columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"])

id
station
date
prcp
tobs


In [ ]:
### Second Try
# engine1.execute(Stations.__tablename__ ,'CSV_cleaned/stations_clean.csv')
def pop(engine,table, csv):

    #connect to the database
    conn=engine.connect()
    
    #Load CSV into pandas
    df_of_data=pd.read_csv(csv)
    
    #reorient
    data=df_of_data.to_dict(orient='records')
    
    conn.execute(table.delete())
    
    conn.execute(table.insert(),data)
    
pop(engine1, Stations.__tablename__ ,'CSV_cleaned/stations_clean.csv')